In [1]:
#import libraries
import pystan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set()

In [35]:
data = pd.read_csv('shop.csv',encoding='latin-1')

store_dict = { 1: 'Lidl', 2: 'Rewe', 3: 'Aldi', 4: 'Edeka' }

area_dict = { 1:"Mitte",2:"Schoneberg",3:"Neukooln",4:"Kreuzberg",5:"Friedrichshain",
             6:"Prenzlauer Berg",7:"Tiergarten",8:"Alt-Trepof",9:"Wedding",
             10:"Gesundbrunnen",11:"Moabit",12:"Rummelsburg",13:"Lichtenberg" }

type_dict = {1:"Apple",2:"Banana",3:"Tomatoes",4:"Potatoes",5:"Flour",
             6:"Rice",7:"Milk",8:"Butter",9:"Eggs",10:"Chicken"}


In [36]:
data

,Grocery store brand,Area,Apple 1 brand,Apple 1 price,Apple 2 brand,Apple 2 price,Apple 3 brand,Apple 3 price,Banana 1 brand,Banana 1 price,...,Eggs 2 brand,Eggs 2 price,Eggs 3 brand,Eggs 3 price,Chicken 1 brand,Chicken 1 price,Chicken 2 brand,Chicken 2 price,Chicken 3 brand,Chicken 3 price
0,1,1,Royal Gala,2.49,Ambriosa,2.49,Normal red apples,1.40,Normal banana,1.09,...,Bodenhaltung (Free Land),2.020,Bio eggs no brand,3.300,Land Junker,6.48,NaN,0.10,NaN,0.10
1,1,1,Royal Gala,2.49,Ambriosa,2.49,Normal red apples,1.40,Normal banana,1.09,...,Bodenhaltung (Free Land),2.020,Bio eggs no brand,3.300,Land Junker,6.48,NaN,0.10,NaN,0.10
2,3,2,Honeycrunch,2.72,Braeburn,1.39,South African,2.99,No brand,1.09,...,Bio Eier,3.300,Eier aus Bodenhaltung,1.290,Friches,6.98,Frisches,7.48,NaN,NaN
3,4,7,Royal Gala,1.99,Pink Lady,2.99,Braeburn,1.99,Normal banana,1.09,...,Big eggs,2.380,Bio eggs,4.980,No brand,6.98,NaN,NaN,NaN,NaN
4,4,4,Kissed By Nature,1.99,Pink Lady,2.93,Edeka,1.99,Edeka,0.88,...,Luisenhof,3.380,Gut & Gunstig,1.550,Stolle,9.99,Biofino,25.99,Meine Fleischerei,9.99
5,3,6,Apfel Evelina,1.79,Apfel Tenroy/Royal Gala,1.39,Apfel rot Honeycrunch,2.74,Gut Bio Fairtrade Bio Bananen,1.65,...,Eier aus Freilandhaltung,2.030,Eier aus Bodenhaltung,1.550,Gut Bio Hahn-Brustfilet,19.99,meine MetzGerei Hahnchen Brustfilet Teilstuck,6.48,NaN,NaN
6,1,1,Bio Organic,3.13,no brand,1.00,no brand,1.40,Fairglobe,1.69,...,Eier Aus Bodenhaltung,1.550,Eier Aus Freidlandhaltung,2.030,Landjunker,6.98,NaN,NaN,NaN,NaN
7,4,1,Granny Smith,1.99,Golden Deli,1.99,Braeburn,1.99,Bio Bananen,1.99,...,Gut & Gunstig,2.030,Edeka,4.190,Le Gaulois,13.90,Friki,12.99,NaN,NaN
8,3,3,Gutbio elstar,4.38,Kissedbynature evelina,1.99,Apfel pink lady,2.79,Bio fairtrade bananen,1.69,...,Gutbio,2.800,Gutbio A,3.300,Gutbio,7.00,Maine metzgerei,6.99,Fair&gut,8.00
9,2,4,Rewe beste wahl,1.52,Pink lady,3.98,Regional,1.99,Chicquita,1.99,...,Ja,1.290,Rewe bio,3.200,No brand,6.59,No brand,6.99,Ja,4.88


In [54]:
# For Stan we provide all known quantities as data, namely the observed data
# and our prior hyperparameters.

prices = np.array([10,9,8,7,6,5,4,3,2,1,10,9,8,7,6,5,4,3,2,1])
types = np.array([1,2,2,4,5,6,7,8,1,10,10,9,8,3,6,5,4,3,2,1])
stores = np.array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])

stan_data = {
                'prices': prices,
                'types': types,
                'stores': stores,
                'N': len(prices), 
                'T': len(set(types)),
                'S': len(set(stores)),
                'alpha': 1,  
                'beta': 0.25
            }

In [56]:
# We have to tell Stan what data to expect, what our parameters are and what
# the likelihood and prior are. Since the posterior is just proportional to
# the product of the likelihood and the prior, we don't distinguish between
# them explicitly in the model below. Every distribution we specify is
# automatically incorporated into the product of likelihood * prior.

stan_code = """

// The data block contains all known quantities - typically the observed
// data and any constant hyperparameters.
data {  

    int<lower=1> N;           // number of observed prices collected
    int<lower=1> T;           // number of different types of products
    int<lower=1> S;           // number of different types of stores
    
    real<lower=0> prices[N];  // data collected
    int types[N];             // data collected
    int stores[N];             // data collected
    
    real<lower=0> alpha;      // fixed prior hyperparameter
    real<lower=0> beta;       // fixed prior hyperparameter
    
}

// The parameters block contains all unknown quantities - typically the
// parameters of the model. Stan will generate samples from the posterior
// distributions over all parameters.
parameters {

    real base_price[T]; //there should only be as many base prices as there are product types
    real multiplier_store[S]; //there should be as many store multipliers as there are stores
    real<lower=0> lambda;
    real<lower=0> sigma2;
    
}


// The model block contains all probability distributions in the model.
// This of this as specifying the generative model for the scenario.
model {

    lambda ~ gamma(alpha, beta);  
    sigma2 ~ inv_gamma(alpha, beta);
    
    //need to assign different lambda, alpha, and beta for different multipliers in a way that makes sense
    
    for(i in 1:N) {
        base_price[types[i]] ~ exponential(lambda);
        multiplier_store[stores[i]] ~ exponential(lambda);
        prices[i] ~ normal(base_price[types[i]] * multiplier_store[stores[i]], sqrt(sigma2));
  }
  
}

"""

In [57]:
stan_model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ff5bf798411e029b2ee5bae2c187be6a NOW.


In [58]:
# Fit the model to the data. This will generate samples from the posterior over
# all parameters of the model.

stan_results = stan_model.sampling(data=stan_data)
print(stan_results)

RuntimeError: Initialization failed.

## Analysis
The spread of the multiplier distributions will determine the impact.

//transformed parameters {
//    vector[N] observed_price;
//    observed_price = base_price[types[N]] * multiplier_store[stores[N]];
//}

In [ ]:
# Finally, we can extract the samples generated by Stan so that we
# can plot them or calculate any other functions or expected values
# we might be interested in.

posterior_samples = stan_results.extract()
plt.figure(figsize=(12, 6))
plt.hist(posterior_samples['lambda'], bins=50, density=True)
plt.title('Sampled posterior probability density for lambda')
print(
    "Posterior 95% confidence interval for lambda:",
    np.percentile(posterior_samples['lambda'], [2.5, 97.5]))
plt.show()